In [ ]:
# arch -x86_64 brew install <package>

In [1]:
import pandas as pd

df = pd.read_csv('dap_history.csv')
print(df.shape)
print(df.dtypes)
df.head()

(786741, 9)
ednum            int64
happening       object
ageinc          object
siteid          object
abstn6mths      object
agereg          object
abstnlongest    object
agefirst        object
backgroundid     int64
dtype: object


/Users/weiching/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,2,5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ednum,happening,ageinc,siteid,abstn6mths,agereg,abstnlongest,agefirst,backgroundid
0,457814,Post partum contributed to depression and incr...,33,ArapahoeHouse,NaN,33,NaN,15,14264
1,732698,NaN,8,ArapahoeHouse,6 months,25,8 years,17,30582
2,1319897,NOTHING DIFFERENT,21,ArapahoeHouse,1 month,18,3 months,12,36136
3,311139,NOTHING,21,ArapahoeHouse,3 months,21,3 months,16,29947
4,988020,NaN,NaN,ArapahoeHouse,NaN,NaN,NaN,18,28172


### abstn6mths

50 unique values cover 96.1%

In [40]:
df['abstn6mths'].value_counts()[:50]

1 month            17779
6 months           15328
2 months           14435
3 months           11883
2 Weeks            11581
1 Week              8620
1 day               6330
5 months            6192
3 days              6072
4 months            5917
3 Weeks             4649
2 days              3291
5 days              2609
4 days              1435
none                 950
0                    940
1 week               775
6 days               765
40 days              634
5-6 months           566
2 weeks              523
a few days           512
3 1/2 month          438
3 weeks              433
unknown              427
last 12 hours        404
2 1/2 months         361
FOUR MONTHS          344
9 days               334
did not specify      330
3 WEEKS              329
7 weeks              310
None                 278
drand once in t      261
1.5 months           233
2 WEEKS              220
21 days              215
Unknown              213
0 days               187
not sure a few       184


Variations dealt with

- '6 Days'
- '3 1/2 - 4 months'

Variations to deal with

- 'since 4/26/2009'

Other things to consider

- How to speed up processing 
- How to deal with no units

#### N

In [224]:
def text2int(textnum, numwords={}):
    try:
        return float(textnum)
    except:
        if not numwords:
            units = {
                "zero": 0,
                "a half": 0.5,
                "half": 0.5,
                "one": 1,
                "a": 1,
                "two": 2,
                "three": 3, 
                "four": 4, 
                "five": 5, 
                "six": 6, 
                "seven": 7, 
                "eight": 8,
                "nine": 9, 
                "ten": 10, 
                "eleven": 11, 
                "twelve": 12, 
                "thirteen": 13, 
                "fourteen": 14, 
                "fifteen": 15,
                "sixteen": 16, 
                "seventeen": 17, 
                "eighteen": 18, 
                "nineteen": 19
            }
            tens = ["", "", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]

            scales = ["hundred", "thousand", "million", "billion", "trillion"]

            numwords["and"] = (1, 0)
            for key, value in units.items():      numwords[key] = (1, value)
            for idx, word in enumerate(tens):     numwords[word] = (1, idx * 10)
            for idx, word in enumerate(scales):   numwords[word] = (10 ** (idx * 3 or 2), 0)

        current = result = 0
        for word in textnum.split():
            if word not in numwords:
                raise Exception("Illegal word: " + word)

            scale, increment = numwords[word]
            current = current * scale + increment
            if scale > 100:
                result += current
                current = 0

        return result + current

print("one and a half: ", text2int("one and a half"))
print("one and half: ", text2int("one and half"))
print("a half: ", text2int("one and a half"))
print("one and a half thousand: ", text2int("one and half thousand"))
print("one hundred and twenty: ", text2int("one hundred and twenty"))
print("1: ", text2int("1"))

one and a half:  2.5
one and half:  1.5
a half:  2.5
one and a half thousand:  1500.0
one hundred and twenty:  120
1:  1.0


In [324]:
# New pipeline
import re

kw_duration = {
    'hour':['hour','hours'],
    'day':['day','days'],
    'week':['week','weeks','wks'],
    'month':['mont','month','months','montths'],
    'year':['year','years']
}
    
def search(values, searchFor): # search if dictionary value contains certain string --> return key, else return string
    for k in values:
        for v in values[k]:
            if str(searchFor) == v:
                return 1, k, ''
    return 0, searchFor, searchFor

def convert_to_day(number, unit): 
    number = float(number)
    unit_conversion = {
        'hour': number / 24,
        'day': number,
        'week': number * 7,
        'month': number * 30,
        'year': number * 365
    }
    return unit_conversion[unit]

def convert_half(n_string):
    return float(n_string.replace('1 / 2','.5').replace(' ',''))


def transform_duration(txt):
    print('\nTransform: ', txt)
    # 1. Add space between numeric and alphabet
    txt = re.sub(r"([0-9]+(\.[0-9]+)?)",r" \1 ", txt).strip()
    print(txt)
    # 2. Detect the number of units
    word_list = txt.split()
    word_list = [search(kw_duration, w) for w in word_list]
    n_units = sum([x[0] for x in word_list if x[0]==1])
    unit_string = ' '.join([x[1] for x in word_list if x[0]==1])
    left_string = ' '.join([x[2] for x in word_list if x[0]==0])
    print('n_units: ', n_units)
    print('unit_string: ', unit_string)
    print('left_string: ', left_string)
    print('...')
    
    # 2a. No units --> unidentified
    if n_units == 0:
        print('0 unit')
        # '128' --> (0, 'unidentified')
        return 0.0, 'unidentified' 
    
    # 2b. 1 unit --> check if remaining string is float
    elif n_units == 1: 
        print('1 unit')
        try: # if successful, return N and unit
            # '6 months' --> (6.0, 'month')
            return float(left_string), unit_string 
        
        except: # if not successful, split the string based on `-` or `to `
            chunk_list = re.split('-|to', left_string)

            try: # try float on each chunk. if successful, take average
                # '1/2 months' --> (0.5, 'month')
                return sum([convert_half(x) for x in chunk_list]) / len(chunk_list), unit_string
            
            except: # if not successful, split the string based on `and ` and sum up
                # '1 and 1/2 months' --> (1.5, 'month')
                chunk_list = re.split('and',''.join(chunk_list))
                try:
                    return sum([convert_half(x) for x in chunk_list]), unit_string
                except: 
                    try: # convert number as words to number int
                        return sum([sum([text2int(x) for x in phrase.replace('a half', 'half').split()]) for phrase in chunk_list]), unit_string 
                        
                    except: # txt contains other words
                        print('text2int mapper fails')
                        return 0.0, 'unidentified'
    
    # 2c. >1 unit
    else:
        print('>1 unit')
        chunk_list = re.split('-|to ', txt)
        chunk_list = [[search(kw_duration, w) for w in chunk.split()] for chunk in chunk_list]
        n_units = [sum([x[0] for x in word_list if x[0]==1]) for word_list in chunk_list]
        unit_string = [' '.join([x[1] for x in word_list if x[0]==1]) for word_list in chunk_list]
        left_string = [' '.join([x[2] for x in word_list if x[0]==0]) for word_list in chunk_list]
        print('n_units: ', n_units)
        print('unit_string: ', unit_string)
        print('left_string: ', left_string)
        print('...')
        
        if sum(n_units) / len(n_units) != 1: # >1 unit per chunk
            return 0.0, 'unidentified'
        else: # 1 unit per chunk
            try: # standardise unit to day
                return sum([convert_to_day(x[0], x[1]) for x in zip(left_string, unit_string)]) / len(left_string), 'day'
            except: # txt contains other words
                return 0.0, 'unidentified'
            

# search(kw_duration, 'months')
# standardise_unit(7, 'week')


# Cases

# No units
# transform_duration('NaN') # (0.0, 'unidentified')
# transform_duration('128') # (0.0, 'unidentified')

# 1 unit
# transform_duration('2 days') # (2.0, 'day')
# transform_duration('2 to 4 days') # (3.0, 'day')
# transform_duration('11/2 day') # (1.5, 'day')
# transform_duration('3 1/2 - 4 months') # (3.75, 'month')
# transform_duration('1 and 1/2 day') # (1.5, 'day')
# transform_duration('one and a half day') # (1.5, 'day')
# transform_duration('1 and a half day') # (1.5, 'day')
# transform_duration('one and a half thousand days') # (0.0, 'unidentified')

# 2 units
# transform_duration('1 day to 2 weeks') # (7.5, 'day')

# Not considered
# transform_duration('6-months') # (0.0, 'unidentified')


In [317]:
df_abstn6mths = pd.DataFrame(data=df['abstn6mths'].unique(), columns=['abstn6mths'])
df_abstn6mths.head()

,abstn6mths
0,NaN
1,6 months
2,1 month
3,3 months
4,1 Week


In [325]:
# df_abstn6mths['abstn6mths_N'] = df_abstn6mths.apply(lambda x: transform_duration(x['abstn6mths']), axis=1)
# df_abstn6mths['abstn6mths_N'][:10]

In [8]:
df[df['abstn6mths'].str.contains("mo", na=False)]['abstn6mths'].value_counts()

#### Units

### abstnlongest


70 unique values cover 95.7%

Length of abstinence period

In [16]:
df['abstnlongest'].value_counts()

2 years          13882
1 year           13247
2 months          8983
6 months          8884
1 month           8847
                 ...  
15-18 days           1
I YEAR               1
10 y                 1
6 WEEKS              1
I don't know.        1
Name: abstnlongest, Length: 370, dtype: int64

### happening

2000 unique values cover 96.8%

In [13]:
df_happening = pd.DataFrame(data=df['happening'].unique(), columns=['happening'])
df_happening.head()

,happening
0,Post partum contributed to depression and incr...
1,NaN
2,NOTHING DIFFERENT
3,NOTHING
4,NO LONGER HAD OVER THE ROAD TRUCK DRIVING JOB ...


In [14]:
pwd

'/home/weiching/weiching-hol'

In [15]:
import stressors

In [16]:
config = {
    'embedding': '/home/weiching/Dropbox (Holmusk)/NLP - Wei Ching/GoogleNews-vectors-optimized.bin'
}

model = stressors.label(config)

In [17]:
data = []
for x in df_happening[['happening']].values.tolist():
    data.append([None,x[0]])

In [18]:
data[:20]

[[None, 'Post partum contributed to depression and increase in drinking.'],
 [None, nan],
 [None, 'NOTHING DIFFERENT'],
 [None, 'NOTHING'],
 [None,
  'NO LONGER HAD OVER THE ROAD TRUCK DRIVING JOB AND WAS HOME EVERY EVENING'],
 [None, 'CLIENT STATES IS ONLY A SOCIAL DRINKER'],
 [None, 'IN THE SERVICE AND SEEING PEOPLE KILLED, BLOWN UP'],
 [None, "it appears clt. doesn't want to give much information"],
 [None, 'Brother was murdered'],
 [None, 'PARTIES WITH FRIENDS AT 16'],
 [None, 'FAMILY PROBLEMS'],
 [None, 'NOTHING REALLY'],
 [None, 'nothing'],
 [None, 'CLT STS DID NOT KNOW'],
 [None, 'father passed away, stressed'],
 [None, 'CROWD CLT HUNG OUT WITH'],
 [None, 'father committed suicide'],
 [None, 'went to college'],
 [None, 'Loss of good jobs and  alot of time on  his hand.'],
 [None,
  "A lot of responsibility helping to raise younger brother because Mom was a single and Dad wans't rea"]]

In [19]:
output, flags = model.generate(data)

print('\nDefault output')
print(output[:20])

print('\nValidated flag')
print(flags[:20])

print('\nOutput labels')
print(model.convert(output[:20]))


Default output
[[], [], [], [], [], [], [], [], [13], [], [], [], [], [], [13], [], [13], [], [64, 66], []]

Validated flag
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Output labels
[[], [], [], [], [], [], [], [], ['Bereavement'], [], [], [], [], [], ['Bereavement'], [], ['Bereavement'], [], ['Unstable job (changing jobs often)', 'Unemployment'], []]


In [28]:
df_happening['default_output'] = output
df_happening['validated_flag'] = flags
df_happening['output_labels'] = model.convert(output)

In [33]:
df_merged = pd.merge(df, df_happening, on=['happening','happening'])
df_merged.head()

,ednum,happening,ageinc,siteid,abstn6mths,agereg,abstnlongest,agefirst,backgroundid,default_output,validated_flag,output_labels
0,457814,Post partum contributed to depression and incr...,33,ArapahoeHouse,NaN,33,NaN,15,14264,[],0.0,[]
1,732698,NaN,8,ArapahoeHouse,6 months,25,8 years,17,30582,[],0.0,[]
2,988020,NaN,NaN,ArapahoeHouse,NaN,NaN,NaN,18,28172,[],0.0,[]
3,687950,NaN,30,ArapahoeHouse,NaN,22,NaN,22,32263,[],0.0,[]
4,127936,NaN,30,ArapahoeHouse,NaN,22,NaN,22,32263,[],0.0,[]


In [35]:
df_merged.to_csv('dap_history_labels.csv',index=False)

In [88]:
df_labels = pd.read_csv('dap_history_labels.csv')

/Users/weiching/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,2,5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [91]:
print(df_labels.shape)

(786741, 12)


In [90]:
df_labels['default_output'].value_counts()

[]          769423
[11]          4316
[13]          2662
[56]          2057
[66]           967
             ...  
[45]             1
[44]             1
[13, 51]         1
[11, 66]         1
[60]             1
Name: default_output, Length: 74, dtype: int64

In [89]:
df_labels['default_output'].value_counts(normalize=True)

[]          0.977988
[11]        0.005486
[13]        0.003384
[56]        0.002615
[66]        0.001229
              ...   
[45]        0.000001
[44]        0.000001
[13, 51]    0.000001
[11, 66]    0.000001
[60]        0.000001
Name: default_output, Length: 74, dtype: float64

In [95]:
df_labels_unique = df_labels[['happening', 'default_output', 'output_labels']].drop_duplicates()

In [122]:
df_labels_unique['output_len'] = df_labels_unique.apply(lambda x:len([i for i in x['default_output'].replace('[','').replace(']','').split(',') if i]), axis=1)
df_labels_unique

,happening,default_output,output_labels,output_len
0,Post partum contributed to depression and incr...,[],[],0
1,NaN,[],[],0
673136,NOTHING DIFFERENT,[],[],0
673137,NOTHING,[],[],0
673293,NO LONGER HAD OVER THE ROAD TRUCK DRIVING JOB ...,[],[],0
...,...,...,...,...
786437,Pt. didn't explain.,[],[],0
786559,loss of mother,[],[],0
786684,felt lonely,[],[],0
786690,school and family problems,[],[],0


In [130]:
df_labels_unique[df_labels_unique.output_len == 0].iloc[:50]

,happening,default_output,output_labels,output_len
0,Post partum contributed to depression and incr...,[],[],0
1,NaN,[],[],0
673136,NOTHING DIFFERENT,[],[],0
673137,NOTHING,[],[],0
673293,NO LONGER HAD OVER THE ROAD TRUCK DRIVING JOB ...,[],[],0
673294,CLIENT STATES IS ONLY A SOCIAL DRINKER,[],[],0
673295,"IN THE SERVICE AND SEEING PEOPLE KILLED, BLOWN UP",[],[],0
673299,it appears clt. doesn't want to give much info...,[],[],0
673302,PARTIES WITH FRIENDS AT 16,[],[],0
673303,FAMILY PROBLEMS,[],[],0


In [121]:
len([x for x in df_labels_unique['default_output'].iloc[-1].replace('[','').replace(']','').split(',') if x])

1